In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

/usr/local/lib/python3.10/dist-packages/yfinance/base.py:48: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  _empty_series = pd.Series()


In [ ]:
def calc_ema(data, window):
    return data['Close'].ewm(span=window, adjust=False).mean()

In [ ]:
def calc_rsi(data, window):
 delta= data.diff()
 gain= delta.where(delta>0,0)
 loss= -delta.where(delta<0,0)
 avg_gain= gain.rolling(window=14).mean()
 avg_loss= loss.rolling(window=14).mean()
 rs= avg_gain/avg_loss
 rsi= 100-(100/(1+rs))
 return rsi

In [ ]:
def generate_signals(data, ema_window, rsi_window, rsi_overbought, rsi_oversold):
    signals = pd.DataFrame(index=data.index)
    signals['EMA'] = calc_ema(data, ema_window)
    signals['RSI'] = calc_rsi(data, rsi_window)
    # Generate Signals
    signals['signal']= 0
    signals['signal'][signals['EMA'] > data['Close']] = 1 #long
    signals['signal'][signals['EMA'] < data['Close']] = -1 #short
    signals['signal'][(signals['RSI'] > rsi_overbought) & (signals['signal'] == 1)] = 0  # exit long
    signals['signal'][(signals['RSI'] < rsi_oversold) & (signals['signal'] == -1)] = 0  # exit short

    return signals


In [ ]:
def backtest_strategy(data, signals, initial_capital=50001, compound=True):
    positions = pd.DataFrame(index=data.index)
    positions['Position'] = signals['Signal']
    positions['Stock'] = 100 * positions['Position']  # Assume 100 shares per trade

    # Calculate daily portfolio value
    portfolio = positions.multiply(data['Close'], axis=0)
    pos_diff = positions.diff()

    # Add cash holdings to the portfolio
    portfolio['Cash'] = initial_capital - (pos_diff['Stock'] * data['Close']).cumsum()

    # Calculate total portfolio value
    portfolio['Total'] = portfolio['Stock'] + portfolio['Cash']

    # Calculate daily returns
    portfolio['Returns'] = portfolio['Total'].pct_change()

    if compound:
        portfolio['Returns'] = (1 + portfolio['Returns']).cumprod() - 1

    return portfolio
